In [1]:
##############################################
import sys,os
import numpy as np
import pylab as py
##############################################
import pylab as py
from  matplotlib import rc
import matplotlib.pyplot as plt
%matplotlib inline  
%config InlineBackend.figure_format = 'retina' 
##############################################
PATH='../python/'
sys.path.append(PATH)  # path of the JAMLIB python folder is located
from corelib import JAMLIB
##############################################

# Code for LHAPDF grids

## JAMLIB testing

In [2]:
def testjam(x,Q2):

  print 'alphaS     = ',jamFF.get_alphaS(Q2)
  print 'no. of posteriors = ',jamFF.npos
  print 'xFF(ipos=0) = ',jamFF.get_XF(0,'up',x,Q2)

  xgrid = jamFF.get_Xgrid()
  xmin = np.min(xgrid)
  xmax = np.max(xgrid)
  print 'Xmin,Xmax: ', xmin,xmax
  print xgrid 

  Q2grid = jamFF.get_Q2grid()
  Q2min = np.min(Q2grid)
  Q2max = np.max(Q2grid)
  print 'Q2min,Q2max: ', Q2min,Q2max
  print Q2grid

## Writes .info file

In [3]:
def writeinfo():
  H=[]
  H.append('SetDesc: '+desc)
  H.append('SetIndex: %i'%index)
  H.append('Authors: '+auth)
  H.append('Reference: '+ref)
  H.append('Format: lhagrid1')
  H.append('DataVersion: %i'%ver)

  H.append('Particle: %i'%particle)
  H.append('Flavors: '+str(flavors))
  H.append('OrderQCD: %i'%ordQCD)
  H.append('ForcePositive: 1')
  flavscheme = 'variable' # used for any variable flavor number scheme, massive or not
  H.append('FlavorScheme: '+flavscheme)
  H.append('NumFlavors: %i'%nfl)

  if errortype=='MC':
    nsets = nsamples+1   # number of posteriors + average
    errtype = 'replicas'  # LHAPDF-speak for MC method
  elif errortype=='HESSIAN': 
    nsets = 2*nsamples+2  # 2*eigenvalues + central fit
    errtype = 'hessian'
  H.append('NumMembers: %i'%nsets)
  H.append('ErrorType: '+errtype)
  H.append('ErrorConfLevel: %f'%cl)
  H.append('XMin: %e'%xmin)
  H.append('XMax: %e'%xmax)
  H.append('QMin: %e'%Qmin)
  H.append('QMax: %e'%Qmax)

  H.append('MZ: %f'%mZ)
  H.append('MUp: %f'%mu)
  H.append('MDown: %f'%md)
  H.append('MStrange: %f'%ms)
  H.append('MCharm: %f'%mc)
  H.append('MBottom: %f'%mb)
  H.append('MTop: %f'%mt)

  H.append('AlphaS_MZ: %f'%jamFF.get_alphaS(mZ**2))
  H.append('AlphaS_OrderQCD: %f'%ordQCD)
  H.append('AlphaS_Type: ipol')
  H.append('AlphaS_Qs: '+str(Qgrid.tolist()))
  alphas = []
  for Q in Qgrid:
    alphas.append(jamFF.get_alphaS(Q**2))
  H.append('AlphaS_Vals: '+str(alphas))
  H.append('AlphaS_Lambda4: %f'%jamFF.get_alphaS(mc**2))
  H.append('AlphaS_Lambda5: %f'%jamFF.get_alphaS(mb**2))

  for l in H: print l 

## Write a single .dat file

In [4]:
def writedat(ipos):
  
  if ipos==0: 
    type='central'
  else:
    type='replica'

  H = []
  H.append('PdfType: '+type)
  H.append('Format: lhagrid1')
  H.append('---')  
  
  for l in H: print l
  
  xvals = ["%10.4e" % x for x in xgrid]
  print ' '.join(xvals)
  Qvals = ["%10.4e" % Q for Q in Qgrid]
  print ' '.join(Qvals)
  flavs = ["%2i" % f for f in flavors]
  print ' '.join(flavs)
  
  npart = len(partons)
  
  full_dist = []
  ix = 0
  iQ = 0
  il = 0
  for x in xgrid: 
    for Q in Qgrid:
      if ipos==0:  # Avreraged PDF
        FF = np.array([0 for n in range(npart)])
        for i in range(nsamples): 
          FF = FF + np.array([jamFF.get_XF(i,f,x,Q**2) for f in partons])
        FF = FF/nsamples
      else:   # ipos-th posterior 
        FF = [jamFF.get_XF(ipos-1,f,x,Q**2) for f in partons]   # <--------- NOTE: for now x*FF(x), Shall we distribute FF(x) as for other PDFs ??  
        #if Q < mb: FF[0] = FF[9] = 0
        #if Q < mc: FF[1] = FF[8] = 0
      if Q < mb: FF[4] = 0
      if Q < mc: FF[3] = 0
      #FF.append([x,Q])
      full_dist.append(FF)
  for l in full_dist: 
    out = ''
    for i in range(len(l)):
      out += '{:12.5e} '.format(l[i])
    print out

  print '---'  

 

## Writes all the .info and .dat files in one folder

In [5]:
def create_LHAPDF_grid(setname):
  
  start = os.getcwd()

  # Creates and navigates to the right folder
  if not os.path.exists(setname):
      os.makedirs(setname)
  os.chdir(setname)

  # Writes the .info file
  stdout = sys.stdout 
  f = open(setname+'.info','w')   
  sys.stdout = f    # <---------- very fragile solution, needs improvement
  writeinfo()
  f.close()
  sys.stdout=stdout

  for igrid in range(nsamples+1):
    gridname = '{}_{:04d}.dat'.format(setname,igrid)
    f = open(gridname,'w')
    sys.stdout = f   # <---------- very fragile solution, needs improvement
    writedat(igrid)
    f.close()
    sys.stdout=stdout
  
  # goes back to the initial folder
  os.chdir(start)



# GRID PRODUCTION

In [6]:
PATH = '../python/'   # path of the JAMLIB pythin folder

# Select JAM analysis
#jam_analysis = "../python/JAM16/FFpion"
jam_analysis = PATH+"JAM16/FFkaon"
jamFF=JAMLIB(jam_analysis)

# Selects LHAPDF set name for given JAM hadron
setname = "JAM16FF_K_Ceven"   # name of teh set as it should appear in LHAPDF
particle = 321  
  #  2212=proton   211=pi+  321=K+
  #
  # "particle" is the hadron to which the PDFs belong, or the FF fragments into, see
  # http://pdg.lbl.gov/2015/reviews/rpp2015-rev-monte-carlo-numbering.pdf
  # For protons, neutrons, use format ±n nr nL nq1 nq2 nq3 nJ
  # E.g, proton = 2212 (u; u; d; spin 1/2 --> nJ=2*s+1)
  # For nuclei, use LZZZAAAI, see point 14 in PDG standard:
  # L nonzero is for hypernuclei, I represent isomer levels with
  # I = 0 the ground state, and Z and A should be self-explanatory.
  

# User-supplied  information for .info file headers               # <-------------  CHECK x*FF or FF for LHAPDF comvention
desc = "'JAMFF16 fragmentation functions of gluons and charge-conjugation-even quarks \
into K+ mesons (identical to K- by charge conjugation). Returns x*FF(x). \
Flavors 1-5: q+ == q+qbar with q=d,u,s,c,b; Flavor 21: gluon. \
mem=0 => average of posteriors; mem=1-200 => FF posteriors.'"
auth = "'[JAM collaboration] Sato N, Ethier J J, Melnitchouk W, Hirai M, Kumano S, Accardi A'"
ref = "'arXiv:1609.00899 - e-mail accardi@jlab.org / sato@jlab.org'"
ver = -1   # -1=beta; 1,2,3, ... official version, and updated/debugged ones if needed
index = 16004
  # index = nnxxx
  # nn (or nnn): assigned by Andy [LHAPDF]; for example 16=JAM fits - this to be discussed with Andy
  # xxx (or xx): up to us --> I would suggest 000-002 for JAM15; 003-004 for JAM16 FF; and so on as needed


########################################################## 
                                                           
     _   _    __  __ _     ___ ____                        
    | | / \  |  \/  | |   |_ _| __ )                       
 _  | |/ _ \ | |\/| | |    | ||  _ \                       
| |_| / ___ \| |  | | |___ | || |_) |                      
 \___/_/   \_\_|  |_|_____|___|____/                       
                                                           
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
                                                           
Authors:                                                   
Nobuo Sato         (Jefferson Lab)                         
Jake Ethier        (College of William and Mary)           
Wally Melnitchouk  (Jefferson Lab)                         
Alberto Accardi    (Hampton University and Jefferson Lab)  
########################################################## 
loading tables [100%]


In [7]:
testjam(0.1,10.)

alphaS     =  0.24654958796
no. of posteriors =  200
xFF(ipos=0) =  0.34776874652
Xmin,Xmax:  0.05 1.0
[ 0.05        0.05959596  0.06919192  0.07878788  0.08838384  0.0979798
  0.10757576  0.11717172  0.12676768  0.13636364  0.1459596   0.15555556
  0.16515152  0.17474747  0.18434343  0.19393939  0.20353535  0.21313131
  0.22272727  0.23232323  0.24191919  0.25151515  0.26111111  0.27070707
  0.28030303  0.28989899  0.29949495  0.30909091  0.31868687  0.32828283
  0.33787879  0.34747475  0.35707071  0.36666667  0.37626263  0.38585859
  0.39545455  0.40505051  0.41464646  0.42424242  0.43383838  0.44343434
  0.4530303   0.46262626  0.47222222  0.48181818  0.49141414  0.5010101
  0.51060606  0.52020202  0.52979798  0.53939394  0.5489899   0.55858586
  0.56818182  0.57777778  0.58737374  0.5969697   0.60656566  0.61616162
  0.62575758  0.63535354  0.64494949  0.65454545  0.66414141  0.67373737
  0.68333333  0.69292929  0.70252525  0.71212121  0.72171717  0.73131313
  0.74090909  0.7505050

In [8]:
# COLLECTS INFO ON JAM FIT

# QCD setup
ordQCD =1  # 1: NLO
nfl = 5
#           Usual PDFs, with q/qbar separation
#           bb  cb  sb  ub  db  d  u  s  c  b  g
#flavors = [-5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 21]
#          DIS/SIA-only analysis --> only q+=q+ + q- is fitted
#          d+ u+ s+ c+ b+  g
flavors = [1, 2, 3, 4, 5, 21]
partons = ['dp','up','sp','cp','bp','g']

# This is for JAM FF or PDF, for others, need to change
errortype = 'MC'      # MC = Monte Carlo method --> nsamples = no. of posteriors
nsamples = jamFF.npos # HESSIAN = Hessian --> nsamples = no. free parameters
cl = 68.3             # confidence level

# Collects info from JAM fits
xgrid = jamFF.get_Xgrid()
xmin = np.min(xgrid)
xmax = np.max(xgrid)
Qgrid = jamFF.get_Q2grid()**0.5
Qmin = np.min(Qgrid)
Qmax = np.max(Qgrid)
mu = 0.
md = 0.
ms = 0.
mc = jamFF.mc2**0.5
mb = jamFF.mb2**0.5
mt = jamFF.mt2**0.5
mZ = jamFF.mZ2**0.5

In [ ]:
writeinfo()

In [ ]:
writedat(1)

In [ ]:
# CREATE THE GRIDS

create_LHAPDF_grid(setname)